# Import Libraries

In [1]:
from swarmintelligence import ParticleSwarmOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/PSO/optimal_params.pkl', 'rb'))
optimal_params

{'particleSize': 25,
 'maxIteration': 100,
 'phi1': 2.6,
 'phi2': 1.7,
 'inertia': 0.5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
pso_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                pso_evaluation_results['image_name'].append(image_name)
                pso_evaluation_results['thresholds'].append(threshold)
                pso_evaluation_results['fitness_function'].append(fitness_function)
                pso_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    pso = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    # Evaluation Metrics
                    st = time.time()
                    particles, best_thresholds = pso.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(pso.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(pso.get_params_training_()['best_fitness_tracking'][-1])

                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                pso_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                pso_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                pso_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                pso_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                pso_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                pso_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                pso_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                pso_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                pso_evaluation_results['Regions'].append(mean_regions)
                pso_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                pso_evaluation_results['Fitness'].append(fitness_values)
                pso_evaluation_results['MSE'].append(list_mse)
                pso_evaluation_results['RMSE'].append(list_rmse)
                pso_evaluation_results['PSNR'].append(list_psnr)
                pso_evaluation_results['SSIM'].append(list_ssim)
                pso_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
12.211474090566005
12.211131163036912
12.211418116190842
12.211418116190842
12.211418116190842
12.211418116190842
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211120689196022
12.211131163036912
12.211418116190842
12.211474090566005
12.211474090566005
12.211474090566005
12.211418116190842
12.211474090566005
12.211131163036912
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211418116190842
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
Mean Fitness = 12.211414957079892
	 AirplaneF16.tiff | Threshold =  3
15.496856729320694
15.493486170404015
15.498855911545927
15.503210052181657
15.503210052181657
15.493756696438219
15.496526075062073
15.502241784438286
15.501611316178057
15.501092051929673
15.5028276682968
15.495470169394252
15.497308010767288
15.498879345952044
15.500414069979893
15.501611316178057
15.5028276682968
15.49

21.56163533062968
21.579060819916364
21.090573345618097
21.579992082935004
21.567685604003334
Mean Fitness = 21.492097861879124
	 Mandrill.tiff | Threshold =  2
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217754102632629
12.217754102632629
12.217754102632629
12.217754102632629
12.217754102632629
12.217632467487256
12.217727299731392
12.217727299731392
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217701316049435
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217754102632629
12.217754102632629
12.217727299731392
12.217727299731392
12.217727299731392
12.217727299731392
12.217754102632629
12.217727299731392
12.217754102632629
Mean Fitness = 12.217739354274597
	 Mandrill.tiff | Threshold =  3
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.26750173434889
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.2791961471

21.024432179341293
20.98402341047845
21.024432179341293
21.022920267281844
21.024702193031583
21.02494765650258
21.01923782383317
21.008791823870283
21.020327863615513
21.024739421716088
21.023198105972373
21.025613403799834
21.02569409352628
21.02375313608652
Mean Fitness = 21.021135613641338


## Visualize Results using DataFrame

In [7]:
pso_evaluation_results_df =pd.DataFrame(pso_evaluation_results)
pso_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[71, 173]",2.863788,12.211415,2802.905751,52.942444,13.654729,0.761606,0.921932,"[[71, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0.9773120880126953, 1.035083293914795, 1.0642...","[12.211474090566005, 12.211131163036912, 12.21...","[2806.3763122558594, 2789.3352966308594, 2797....","[52.97524244640943, 52.81415810775421, 52.8913...","[13.64934454910294, 13.675796383823773, 13.663...","[0.761346872735969, 0.7626303555989788, 0.7620...","[0.9217762274412886, 0.9225501459687783, 0.922..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[69, 127, 182]",3.741464,15.499582,2003.696070,44.762281,15.112634,0.798814,0.949240,"[[69, 255, 255, 255, 182, 255, 255, 255, 255, ...","[2.7189407348632812, 4.1549036502838135, 4.145...","[15.496856729320694, 15.493486170404015, 15.49...","[2025.4875717163086, 2035.226707458496, 2011.5...","[45.005417137454785, 45.11348697959953, 44.850...","[15.065507781479015, 15.04467567791028, 15.095...","[0.8014724864560607, 0.8006019822771487, 0.799...","[0.9491166447656872, 0.9490779154361665, 0.949..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[66, 107, 146, 186]",3.915043,18.295154,1803.906205,42.469753,15.569760,0.809597,0.957117,"[[66, 255, 255, 186, 186, 186, 255, 255, 255, ...","[3.308608055114746, 4.777183532714844, 4.11529...","[18.28971321239836, 18.303217832512797, 18.305...","[1755.440258026123, 1820.9238586425781, 1828.1...","[41.89797439048961, 42.67228443196565, 42.7569...","[15.686943069304448, 15.527885745802514, 15.51...","[0.8032224423026956, 0.8129096078144143, 0.813...","[0.9581241353353094, 0.9569149981534493, 0.957..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[62, 93, 127, 159, 188]",3.565472,20.869816,1678.483585,40.959446,15.885823,0.809456,0.961627,"[[93, 255, 255, 188, 188, 188, 255, 255, 255, ...","[3.9858248233795166, 3.915440320968628, 3.0153...","[20.86363224736595, 20.890423432738697, 20.887...","[1796.3577499389648, 1698.656509399414, 1744.1...","[42.38346080653354, 41.21476081938865, 41.7628...","[15.586875288972667, 15.82974793154887, 15.715...","[0.8211592361713729, 0.8109573658165506, 0.818...","[0.9590705718128388, 0.9609439042622905, 0.960..."
4,Lena.png,2,kapur_entropy,max,"[97, 165]",3.819466,12.346004,2032.814727,45.086448,15.049941,0.631538,0.891580,"[[165, 165, 165, 165, 165, 165, 165, 165, 255,...","[4.1640589237213135, 3.530796766281128, 3.9684...","[12.346128939025652, 12.34523250692625, 12.346...","[2018.693244934082, 2054.2718811035156, 2044.6...","[44.92987029732094, 45.32407617484901, 45.2182...","[15.080100311040322, 15.004224393043204, 15.02...","[0.6320976640899426, 0.632586006760257, 0.6307...","[0.8915252260170561, 0.8917118147125861, 0.891..."
5,Lena.png,3,kapur_entropy,max,"[82, 127, 176]",3.745800,15.314227,1195.235677,34.569558,17.357586,0.705197,0.932401,"[[176, 176, 176, 176, 176, 176, 176, 176, 176,...","[2.727169990539551, 4.362809419631958, 3.10130...","[15.311669019511212, 15.315658293887235, 15.31...","[1133.241600036621, 1169.1116981506348, 1199.4...","[33.66365399116117, 34.19227541639536, 34.6331...","[17.587578522602946, 17.45224354776179, 17.340...","[0.709713635760836, 0.7049172384642527, 0.7061...","[0.9309401938755409, 0.9333699547935279, 0.934..."
6,Lena.png,4,kapur_entropy,max,"[73, 112, 151, 186]",4.246963,17.981030,788.544443,28.055901,19.177959,0.747804,0.953574,"[[186, 186, 186, 186, 186, 186, 186, 186, 186,...","[5.8093016147613525, 2.969053030014038, 5.0785...","[17.980045370374395, 17.97709380492285, 17.977...","[718.9938468933105, 748.7368545532227, 772.446...","[26.814060619259266, 27.363056381793733, 27.79...","[19.563551871355372, 19.387511502386403, 19.25...","[0.745487806833921, 0.7562970575603356, 0.7588...","[0.9501877681219151, 0.9526193193845423, 0.955..."
7,Lena.png,5,kapur_en

# Save results

In [8]:
pickle.dump(pso_evaluation_results_df, open('results/evaluation/PSO/PSO_kapur_df.pkl', 'wb'))
pickle.dump(pso_evaluation_results, open('results/evaluation/PSO/PSO_kapur_dict.pkl', 'wb'))